# # China Real Estate Demand Prediction — Pipeline Skeleton
#
# **Last generated:** 2025-10-06 07:47 UTC
#
# This notebook/script is a lean, safe baseline skeleton tailored to the Kaggle
# competition *"Real Estate Demand Prediction"*.
#
# ### What you get
# - Robust **IO setup** targeting Kaggle input paths
# - **Custom competition metric** (two-stage MAPE-based)
# - **Leakage-safe** rolling time-grouped cross-validation
# - Minimal **feature factory** with lags/rollings (extend here)
# - **Naïve** baseline (strong Stage-1 shield)
# - **LightGBM Tweedie** model scaffold + optional XGB/CatBoost hooks
# - **Blending + clipping** for metric safety
# - **Submission writer** preserving `test.csv` row order
#
# > Notes:
# > - Extend features in the marked sections. Keep temporal embargo ≥ max lag.
# > - If the official epsilon/edge-case handling differs, update the metric function accordingly.
#
# ---

In [3]:
import os, gc, sys, math, warnings, itertools, json
from dataclasses import dataclass
from typing import List, Tuple, Dict, Optional

import numpy as np
import pandas as pd

warnings.filterwarnings("ignore")

try:
    import lightgbm as lgb
except Exception as e:
    lgb = None

try:
    import xgboost as xgb
except Exception as e:
    xgb = None

try:
    from catboost import CatBoostRegressor, Pool as CatPool
except Exception as e:
    CatBoostRegressor, CatPool = None, None

In [4]:
class CFG:
    seed = 42
    n_folds = 5
    embargo_months = 3             # >= max lag window used in features
    use_target_lags = True         # if True, recursive inference to populate target lags
    use_tweedie = True             # main LightGBM objective
    lgb_params = dict(
        objective = "tweedie",
        tweedie_variance_power = 1.3,  # tune 1.3~1.7
        metric = "mae",
        num_leaves = 63,
        max_depth = 8,
        learning_rate = 0.05,
        feature_fraction = 0.6,
        bagging_fraction = 0.8,
        bagging_freq = 1,
        min_data_in_leaf = 128,
        lambda_l1 = 0.0,
        lambda_l2 = 5.0,
        n_estimators = 3000,
        verbose = -1,
        random_state = seed
    )

    data_dir = "/kaggle/input/china-real-estate-demand-prediction"
    out_dir  = "/kaggle/working"
    target_col = "amount_new_house_transactions"  # train target column name
    id_col     = "id"
    month_col  = "month"
    sector_col = "sector"
    sector_int_col = "sector_int"
    # files
    files = dict(
        new_house="train/new_house_transactions.csv",
        new_house_nb="train/new_house_transactions_nearby_sectors.csv",
        pre_owned="train/pre_owned_house_transactions.csv",
        pre_owned_nb="train/pre_owned_house_transactions_nearby_sectors.csv",
        land="train/land_transactions.csv",
        land_nb="train/land_transactions_nearby_sectors.csv",
        poi="train/sector_POI.csv",
        search="train/city_search_index.csv",
        city="train/city_indexes.csv",
        test="test.csv",
        sample="sample_submission.csv"
    )

In [5]:
def set_seed(seed:int=42):
    import random
    random.seed(seed); np.random.seed(seed)

def month_to_timestamp(m: str) -> pd.Timestamp:
    """
    Parse month strings like '2019 Jan' to Timestamp at month-end.
    """
    # there might be localized month abbreviations; try a few formats
    for fmt in ["%Y %b", "%Y-%m", "%b %Y"]:
        try:
            return pd.to_datetime(m, format=fmt) + pd.offsets.MonthEnd(0)
        except Exception:
            pass
    # fallback
    return pd.to_datetime(m) + pd.offsets.MonthEnd(0)

def parse_sector(val) -> int:
    """
    Convert 'sector 3' or 3 -> int 3.
    """
    if pd.isna(val): return np.int16(-1)
    if isinstance(val, (int, np.integer)): return int(val)
    s = str(val).strip().lower()
    for token in ["sector", "_", "-"]:
        s = s.replace(token, " ")
    parts = s.split()
    nums = [p for p in parts if p.isdigit()]
    return int(nums[-1]) if nums else int(float(parts[-1])) if parts else -1

def ensure_cols(df: pd.DataFrame, cols: List[str]):
    for c in cols:
        if c not in df.columns:
            df[c] = np.nan
    return df

def reduce_mem(df: pd.DataFrame) -> pd.DataFrame:
    for col in df.columns:
        if pd.api.types.is_float_dtype(df[col]):
            df[col] = pd.to_numeric(df[col], downcast="float")
        elif pd.api.types.is_integer_dtype(df[col]):
            df[col] = pd.to_numeric(df[col], downcast="integer")
    return df


In [6]:
def load_csv(path: str, dtype=None) -> pd.DataFrame:
    full = os.path.join(CFG.data_dir, path)
    if not os.path.exists(full):
        print(f"[WARN] Missing file: {full}")
        return pd.DataFrame()
    df = pd.read_csv(full, dtype=dtype)
    return df

def load_all():
    dfs = {}
    for k, rel in CFG.files.items():
        dfs[k] = load_csv(rel)
    return dfs

In [7]:
def normalize_month_sector(df: pd.DataFrame) -> pd.DataFrame:
    """
    Standardize month -> Timestamp month-end; sector -> int.
    Some CSVs might miss either column or encode differently; handle robustly.
    """
    df = df.copy()
    if CFG.month_col in df.columns:
        df[CFG.month_col] = df[CFG.month_col].astype(str).str.strip().apply(month_to_timestamp)
    else:
        # Try to split from 'id' pattern 'YYYY Mon_sector n'
        if CFG.id_col in df.columns:
            tmp = df[CFG.id_col].astype(str).str.split("_sector", n=1, expand=True)
            df[CFG.month_col] = tmp[0].str.strip().apply(month_to_timestamp)
            df[CFG.sector_col] = "sector " + tmp[1].str.strip()
    if CFG.sector_col in df.columns:
        df[CFG.sector_int_col] = df[CFG.sector_col].apply(parse_sector).astype("int16")
    else:
        # try parse from 'month' if it accidentally concatenated
        # e.g. '2019 Jan_sector 3' stored under month
        mask = df[CFG.month_col].astype(str).str.contains("sector", case=False, na=False)
        if mask.any():
            ss = df.loc[mask, CFG.month_col].astype(str)
            # split
            mm = ss.str.split("_sector", n=1, expand=True)
            df.loc[mask, CFG.month_col] = mm[0].apply(month_to_timestamp)
            df.loc[mask, CFG.sector_int_col] = mm[1].apply(parse_sector).astype("int16")
        else:
            df[CFG.sector_int_col] = -1
    return df

In [8]:
def add_group_lags(df: pd.DataFrame, cols: List[str], lags: List[int]) -> pd.DataFrame:
    df = df.sort_values([CFG.sector_int_col, CFG.month_col]).copy()
    for c in cols:
        for lag in lags:
            df[f"{c}_lag{lag}"] = df.groupby(CFG.sector_int_col, observed=True)[c].shift(lag)
    return df

def add_group_rollings(df: pd.DataFrame, cols: List[str], windows: List[int], funcs=("mean","median")) -> pd.DataFrame:
    df = df.sort_values([CFG.sector_int_col, CFG.month_col]).copy()
    for c in cols:
        for w in windows:
            gb = df.groupby(CFG.sector_int_col, observed=True)[c]
            for fn in funcs:
                df[f"{c}_roll{w}_{fn}"] = gb.shift(1).rolling(w, min_periods=max(1, w//2)).agg(fn).values
    return df

def build_feature_matrix(dfs: Dict[str, pd.DataFrame]) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Returns: train_df, test_df, features_list
    """
    # Load and normalize
    nh = normalize_month_sector(dfs.get("new_house", pd.DataFrame()))
    nh_nb = normalize_month_sector(dfs.get("new_house_nb", pd.DataFrame()))
    po = normalize_month_sector(dfs.get("pre_owned", pd.DataFrame()))
    po_nb = normalize_month_sector(dfs.get("pre_owned_nb", pd.DataFrame()))
    land = normalize_month_sector(dfs.get("land", pd.DataFrame()))
    land_nb = normalize_month_sector(dfs.get("land_nb", pd.DataFrame()))
    poi = dfs.get("poi", pd.DataFrame()).copy()
    search = dfs.get("search", pd.DataFrame()).copy()
    city = dfs.get("city", pd.DataFrame()).copy()
    test = normalize_month_sector(dfs.get("test", pd.DataFrame()))
    sample = dfs.get("sample", pd.DataFrame()).copy()

    # Canonical index from new_house (target CSV)
    base = nh[[CFG.month_col, CFG.sector_int_col]].drop_duplicates().copy()
    base["year"] = base[CFG.month_col].dt.year.astype("int16")
    base["month_num"] = base[CFG.month_col].dt.month.astype("int8")

    # Merge core numeric columns from new_house
    use_cols_nh = [CFG.target_col,
                   "num_new_house_transactions",
                   "area_new_house_transactions",
                   "price_new_house_transactions",
                   "num_new_house_available_for_sale",
                   "area_new_house_available_for_sale",
                   "period_new_house_sell_through"]
    base = base.merge(nh[[CFG.month_col, CFG.sector_int_col]+[c for c in use_cols_nh if c in nh.columns]],
                      on=[CFG.month_col, CFG.sector_int_col], how="left")

    # Nearby aggregates (if present)
    if not nh_nb.empty:
        suf = "_nearby"
        map_cols = {
            "num_new_house_transactions":"num_new_house_transactions"+suf,
            "area_new_house_transactions":"area_new_house_transactions"+suf,
            "price_new_house_transactions":"price_new_house_transactions"+suf,
            CFG.target_col:"amount_new_house_transactions"+suf,
            "num_new_house_available_for_sale":"num_new_house_available_for_sale"+suf,
            "area_new_house_available_for_sale":"area_new_house_available_for_sale"+suf,
            "period_new_house_sell_through":"period_new_house_sell_through"+suf
        }
        nh_nb = nh_nb.rename(columns={k:v for k,v in map_cols.items() if k in nh_nb.columns})
        base = base.merge(nh_nb[[CFG.month_col, CFG.sector_int_col]+[c for c in map_cols.values() if c in nh_nb.columns]],
                          on=[CFG.month_col, CFG.sector_int_col], how="left")

    # Pre-owned & land (own + nearby), keep a few signals; extend as needed
    def merge_selected(src, prefix):
        if src.empty: return base
        cols = [c for c in src.columns if c not in [CFG.month_col, CFG.sector_col, CFG.sector_int_col]]
        keep = []
        for k in cols:
            if any(x in k for x in ["amount", "price", "area", "num", "planned_building_area", "construction_area", "transaction_amount"]):
                keep.append(k)
        sel = src[[CFG.month_col, CFG.sector_int_col]+keep].copy()
        # rename with prefix to avoid collisions
        sel = sel.rename(columns={c:f"{prefix}_{c}" for c in keep})
        return base.merge(sel, on=[CFG.month_col, CFG.sector_int_col], how="left")

    base = merge_selected(po, "po")
    base = merge_selected(po_nb, "po_nb")
    base = merge_selected(land, "land")
    base = merge_selected(land_nb, "land_nb")

    # POI static features: compress or pass through (skeleton: pass through top-level counts)
    if not poi.empty:
        poi = poi.copy()
        if CFG.sector_col in poi.columns:
            poi[CFG.sector_int_col] = poi[CFG.sector_col].apply(parse_sector).astype("int16")
        keep_poi = [c for c in poi.columns if c not in [CFG.sector_col, CFG.sector_int_col]]
        base = base.merge(poi[[CFG.sector_int_col]+keep_poi], on=CFG.sector_int_col, how="left")

    # City yearly indexes: forward fill within year
    if not city.empty:
        city = city.copy()
        # align to month: join on 'year'
        if "city_indicator_data_year" in city.columns:
            city["year"] = city["city_indicator_data_year"].astype(int)
        elif "year" not in city.columns:
            # best-effort guess
            for c in city.columns:
                if c.lower().endswith("year"):
                    city["year"] = city[c].astype(int); break
        num_cols = [c for c in city.columns if c not in ["year","city_indicator_data_year"]]
        agg = city.groupby("year", as_index=False)[num_cols].mean()
        base = base.merge(agg, on="year", how="left")

    # Minimal calendar encodings
    base["m_sin"] = np.sin(2*np.pi*base["month_num"]/12)
    base["m_cos"] = np.cos(2*np.pi*base["month_num"]/12)

    # Lags/Rollings (extend this list)
    lag_cols = [c for c in [
        CFG.target_col,
        "num_new_house_transactions",
        "area_new_house_transactions",
        "price_new_house_transactions",
        "num_new_house_transactions_nearby",
        "area_new_house_transactions_nearby",
        "price_new_house_transactions_nearby",
        "land_transaction_amount",
        "land_planned_building_area",
    ] if c in base.columns]

    base = add_group_lags(base, lag_cols, lags=[1,2,3,6,12])
    base = add_group_rollings(base, lag_cols, windows=[3,6,12])

    # Sort & memory
    base = base.sort_values([CFG.month_col, CFG.sector_int_col]).reset_index(drop=True)
    base = reduce_mem(base)

    # Split into train rows with target and test rows from test.csv
    test = test.copy()
    if CFG.id_col in test.columns and CFG.month_col not in test.columns:
        # need to split id => month + sector
        tmp = test[CFG.id_col].astype(str).str.split("_sector", n=1, expand=True)
        test[CFG.month_col] = tmp[0].str.strip().apply(month_to_timestamp)
        test[CFG.sector_int_col] = tmp[1].apply(parse_sector).astype("int16")
    test = test[[CFG.id_col, CFG.month_col, CFG.sector_int_col]].copy()

    train_df = base[~base[CFG.target_col].isna()].copy()
    test_df  = base.merge(test, on=[CFG.month_col, CFG.sector_int_col], how="right", suffixes=("",""))

    # Features list
    feats = [c for c in train_df.columns if c not in
             [CFG.target_col, CFG.id_col, CFG.sector_col, CFG.sector_int_col, CFG.month_col, "year", "month_num"]]

    # Zero hints: missing month-sector in train means true 0 => keep as is; lags handle naturally
    return train_df, test_df, feats


In [9]:
def comp_score(y_true: np.ndarray, y_pred: np.ndarray, eps: float = 1e-6) -> float:
    """
    Two-stage custom score described in the Data tab.
    Stage 1: if >30% rows have APE > 1 (100%), score = 0
    Stage 2: compute MAPE on rows with APE <= 1; divide by fraction kept; score = 1 - scaled MAPE.
    """
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    denom = np.maximum(np.abs(y_true), eps)
    ape = np.abs(y_true - y_pred) / denom

    fail_rate = (ape > 1.0).mean()
    if fail_rate > 0.30:
        return 0.0

    mask = ape <= 1.0
    if mask.sum() == 0:
        return 0.0
    mape = ape[mask].mean()
    frac = mask.mean()
    scaled = mape / frac
    return float(1.0 - scaled)

In [10]:
def rolling_time_splits(df: pd.DataFrame, n_folds: int, embargo_months: int) -> List[Tuple[np.ndarray, np.ndarray]]:
    """
    Returns list of (train_idx, valid_idx) using month as time key.
    Embargo period removes last `embargo_months` from train before validation start.
    """
    uniq_months = np.array(sorted(df[CFG.month_col].unique()))
    folds = []
    # leave at least last fold for validation; simple uniform split by month
    month_splits = np.array_split(uniq_months, n_folds + 1)  # n_folds valid blocks; first chunk(s) train start
    # build folds by progressively expanding train, next chunk as valid
    train_months = np.concatenate(month_splits[:-1])
    start = 0
    for i in range(n_folds):
        valid_months = month_splits[i+1]
        # embargo: drop last X train months that are too close to validation
        if embargo_months > 0:
            last_train_cut = valid_months.min() - pd.offsets.MonthEnd(embargo_months)
            tr_mask = (df[CFG.month_col] <= last_train_cut)
        else:
            tr_mask = df[CFG.month_col].isin(np.concatenate(month_splits[:i+1]))
        va_mask = df[CFG.month_col].isin(valid_months)
        tr_idx = df.index[tr_mask].to_numpy()
        va_idx = df.index[va_mask].to_numpy()
        if len(tr_idx) == 0 or len(va_idx) == 0:
            continue
        folds.append((tr_idx, va_idx))
    return folds


In [11]:
def fit_lgbm(X_tr, y_tr, X_va, y_va, params):
    if lgb is None:
        print("[WARN] lightgbm not available; skipping.")
        return None
    train_set = lgb.Dataset(X_tr, label=y_tr)
    valid_set = lgb.Dataset(X_va, label=y_va)
    model = lgb.train(
        params=params,
        train_set=train_set,
        valid_sets=[train_set, valid_set],
        valid_names=["train","valid"],
        num_boost_round=params.get("n_estimators", 3000),
        early_stopping_rounds=200,
        verbose_eval=200
    )
    return model

In [ ]:
def run_training(train_df: pd.DataFrame, feats: List[str]) -> Dict:
    oof = np.zeros(len(train_df), dtype=float)
    models = []
    folds = rolling_time_splits(train_df, CFG.n_folds, CFG.embargo_months)
    print(f"[CV] Folds: {len(folds)}")

    for i, (tr_idx, va_idx) in enumerate(folds, 1):
        tr = train_df.iloc[tr_idx]; va = train_df.iloc[va_idx]
        X_tr, y_tr = tr[feats], tr[CFG.target_col]
        X_va, y_va = va[feats], va[CFG.target_col]

        params = CFG.lgb_params.copy()
        if not CFG.use_tweedie:
            params["objective"] = "mae"  # alternative robust
            params.pop("tweedie_variance_power", None)

        model = fit_lgbm(X_tr, y_tr, X_va, y_va, params=params)
        if model is not None:
            pred = model.predict(X_va, num_iteration=model.best_iteration)
        else:
            # fallback naive if LGBM not available
            pred = tr.groupby(CFG.sector_int_col)[CFG.target_col].transform("last").reindex(va.index).fillna(y_tr.median()).values

        oof[va_idx] = np.clip(pred, 0, None)
        models.append(model)
        score = comp_score(y_va.values, oof[va_idx])
        print(f"[Fold {i}] valid score: {score:.6f}  (n_va={len(va)})")

    overall = comp_score(train_df[CFG.target_col].values, oof)
    print(f"[OOF] Overall score: {overall:.6f}")
    return {"oof": oof, "models": models, "folds": folds, "feats": feats, "oof_score": overall}


In [12]:
def naive_by_sector(train_df: pd.DataFrame, test_df: pd.DataFrame) -> np.ndarray:
    """
    Predict last known target per sector.
    """
    last = (train_df
            .sort_values([CFG.sector_int_col, CFG.month_col])
            .groupby(CFG.sector_int_col)[CFG.target_col]
            .last())
    # map to test
    m = test_df[CFG.sector_int_col].map(last).fillna(train_df[CFG.target_col].median())
    return m.to_numpy()

def blend_safe(pred_main: np.ndarray, pred_naive: np.ndarray, w_main: float = 0.6) -> np.ndarray:
    w_main = float(np.clip(w_main, 0.0, 1.0))
    return np.clip(w_main * pred_main + (1.0 - w_main) * pred_naive, 0, None)

In [ ]:
def infer(models: List, train_df: pd.DataFrame, test_df: pd.DataFrame, feats: List[str]) -> np.ndarray:
    """
    Predict test rows. For skeleton we use average of LGBMs; if no model, fall back to naive.
    """
    if not models or models[0] is None:
        print("[INFO] Using naïve predictions (no models available).")
        return naive_by_sector(train_df, test_df)

    preds = []
    for m in models:
        if m is None: continue
        preds.append(m.predict(test_df[feats], num_iteration=m.best_iteration))
    if not preds:
        return naive_by_sector(train_df, test_df)
    p = np.mean(preds, axis=0)
    return np.clip(p, 0, None)